# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## 1. Загрузка и подготовка данных

In [239]:
# Подключаем все необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Разбиение на обучающую, валидационную и тестовую выборку
from sklearn.model_selection import train_test_split
# Применим кроссвалидацию для повышения качеств обучения
from sklearn.model_selection import cross_val_score

# Масштабируемость модели
from sklearn.preprocessing import StandardScaler

# и для машинного обучения разными способами (по условию мы выбираем линейную регрессию):
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import (
    # Точность модели
    accuracy_score,
    # Матрицы ошибок (для борьбы с дисбалансом)
    confusion_matrix, 
    # Полнота
    recall_score, 
    # Точность
    precision_score, 
    # F1-мера
    f1_score,
    # Метрика AUC-ROC
    roc_auc_score,
    roc_curve,
    # MSE
    mean_squared_error
)

# Контроль выборки
from sklearn.utils import shuffle

In [330]:
# Получаем датасеты
try:
    # Локальный файл
    try:
        df_0 = pd.read_csv('geo_data_0.csv')
        df_1 = pd.read_csv('geo_data_1.csv')
        df_2 = pd.read_csv('geo_data_2.csv')
    # Файл с яндекса
    except:
        df_0 = pd.read_csv('/datasets/geo_data_0.csv')
        df_1 = pd.read_csv('/datasets/geo_data_1.csv')
        df_2 = pd.read_csv('/datasets/geo_data_2.csv')

    # Узнаем, что в них
    df_0.info()
    df_1.info()
    df_2.info()
except:
    print('Отсутствует датасет. Проверьте путь файла')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [241]:
df_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [242]:
df_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [243]:
df_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [244]:
# Проверим на наличие пропущенных значений
df_0.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [245]:
df_1.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

In [246]:
df_2.isna().sum()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64

Все в порядке, теперь можно приступать к обучению и проверку модели

## 2. Обучение и проверка модели

In [247]:
# Убираем лишний признак
df_0 = df_0[['f0', 'f1', 'f2', 'product']]
df_1 = df_1[['f0', 'f1', 'f2', 'product']]
df_2 = df_2[['f0', 'f1', 'f2', 'product']]

In [248]:
# Объединим в один датасет в виде списка для удобства автоматизации
full_df = [df_0, df_1, df_2]

In [249]:
# Эта переменная будет сохранять все RMSE модели для всех регионов 
best_RMSE_for_all_model_averange_for_all_region = []

# А эта будет сохранять все RMSE модели
best_RMSE_for_all_model_averange = []

In [250]:
# Эта - все предсказания модели для всех регионов
best_predict_for_all_model_averange_for_all_region = []

# Аналогично, все предсказания модели
best_predict_for_all_model_averange = []

In [251]:
features = []
target = []

for df in full_df:
    features.append(df.drop('product', axis=1))
    target.append(df['product'])

### 2.1. Разобьем данные на обучающую и валидационную выборки в соотношении 75:25

In [252]:
# Как обычно, будем разделять 75% данных для обучающей выборки и 25% валидационной выборки

features_train = []
features_valid = []
target_train = []
target_valid = []

for i in range(0, 3):
    f_t, f_v, t_t, t_v = train_test_split(features[i], target[i], test_size=(1 - 0.75), random_state=12345)
    features_train.append(f_t)
    features_valid.append(f_v)
    target_train.append(t_t)
    target_valid.append(t_v)

### 2.2. Обучим модель и сделаем предсказания на валидационной выборке, 
### 2.3. Также сохраним предсказания и правильные ответы на валидационной выборки

Попробуем выявлять предсказания и RMSE методом LinearRegression

In [253]:
%%time
# Применяем метод линейной регрессии
model = LinearRegression()

for i in range(0, 3):
    # Обучим модель на тренировочной выборке
    model.fit(features_train[i], target_train[i]) 

    # Получим предсказания модели на валидационной выборке
    predictions_valid = model.predict(features_valid[i]) 

    # Посчитаем значение метрики RMSE на валидационной выборке
    result = mean_squared_error(target_valid[i], predictions_valid) ** 0.5
    
    print('-'*100)
    print('Регион №', i + 1)
    print('RMSE модели линейной регрессии на валидационной выборке:', result)
    print('Предсказание модели:', predictions_valid.mean())
    
    # Сохраним все значения RMSE
    best_RMSE_for_all_model_averange.append(result)
    # И предсказания
    best_predict_for_all_model_averange.append(predictions_valid)
    
print('-'*100)

----------------------------------------------------------------------------------------------------
Регион № 1
RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813
Предсказание модели: 92.59256778438035
----------------------------------------------------------------------------------------------------
Регион № 2
RMSE модели линейной регрессии на валидационной выборке: 0.8930992867756167
Предсказание модели: 68.728546895446
----------------------------------------------------------------------------------------------------
Регион № 3
RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434
Предсказание модели: 94.96504596800489
----------------------------------------------------------------------------------------------------
CPU times: total: 15.6 ms
Wall time: 22.6 ms


У перовго региона RMSE небольшое, но ощутимое. Значит где-то 37 объём запасов в скважине (тыс. баррелей) присутствует отклонение

А у второго - самое минимальное отклонение, можно рассмотреть этот регион как лучший

У третьего - больше отклонений первого региона

Здесь получается, что можно рассмотреть второй регион как минимальное отклонение

А предсказания у второго региона самое низкие остальных

In [254]:
# Сохраним лучшие RMSE всех регионов
best_RMSE_for_all_model_averange_for_all_region.append(best_RMSE_for_all_model_averange[:])
# И предсказания всех регионов
best_predict_for_all_model_averange_for_all_region.append(best_predict_for_all_model_averange[:])

In [255]:
# Очистим, чтобы заполняли новые значения других регионов
best_RMSE_for_all_model_averange.clear()

best_predict_for_all_model_averange.clear()

In [256]:
print(best_predict_for_all_model_averange_for_all_region)
print(best_RMSE_for_all_model_averange_for_all_region)

[[array([ 95.89495185,  77.57258261,  77.89263965, ...,  61.50983303,
       118.18039721, 118.16939229]), array([ 82.66331365,  54.43178616,  29.74875995, ..., 137.87934053,
        83.76196568,  53.95846638]), array([ 93.59963303,  75.10515854,  90.06680936, ...,  99.40728116,
        77.77991248, 129.03241718])]]
[[37.5794217150813, 0.8930992867756167, 40.02970873393434]]


А что если попробуем снвоа выявлять предсказания и RMSE, но с другим методом DecisionTreeRegressor

In [257]:
%%time

# Выявляем RMSE

for i in range(0, 3):
    print('='*100)
    print('Регион №', i + 1)
    print()
    
    best_model = None
    best_RMSE = 10000
    best_depth = 0

    for depth in range(1, 16):
        
        # Инициализируем модель DecisionTreeRegressor с параметром random_state=12345 и max_depth=depth
        model = DecisionTreeRegressor(random_state = 12345, max_depth = depth)

        # Обучим модель на тренировочной выборке
        model.fit(features_train[i], target_train[i])

        # Получим предсказания модели на валидационной выборке
        predictions_valid = model.predict(features_valid[i]) 

        # Посчитаем значение метрики RMSE на валидационной выборке
        RMSE = mean_squared_error(target_valid[i], predictions_valid) ** 0.5

        print('Глубина дерева:', depth, '; RMSE модели:', RMSE, '; Предсказание модели:', predictions_valid.mean())

        if RMSE < best_RMSE:
            best_model = model
            best_RMSE = RMSE
            best_depth = depth
            best_predictions = predictions_valid

    print()
    print("RMSE наилучшей модели на валидационной выборке:", best_RMSE, "Глубина дерева:", best_depth)
    print('Лучшие среднее предсказание модели:', best_predictions.mean())
    
    # Сохраним все значения RMSE
    best_RMSE_for_all_model_averange.append(best_RMSE)
    
    # И предсказания
    best_predict_for_all_model_averange.append(best_predictions)
    
print('='*100)

Регион № 1

Глубина дерева: 1 ; RMSE модели: 40.48057067576246 ; Предсказание модели: 92.57866188386241
Глубина дерева: 2 ; RMSE модели: 39.17407446095702 ; Предсказание модели: 92.48986723373261
Глубина дерева: 3 ; RMSE модели: 38.42907789021468 ; Предсказание модели: 92.441156218743
Глубина дерева: 4 ; RMSE модели: 37.99574447584124 ; Предсказание модели: 92.47660662781237
Глубина дерева: 5 ; RMSE модели: 37.637645696368445 ; Предсказание модели: 92.54680334002089
Глубина дерева: 6 ; RMSE модели: 37.57734774085674 ; Предсказание модели: 92.59749100983761
Глубина дерева: 7 ; RMSE модели: 37.5388716462289 ; Предсказание модели: 92.62808264682461
Глубина дерева: 8 ; RMSE модели: 37.52246994257509 ; Предсказание модели: 92.62631910469075
Глубина дерева: 9 ; RMSE модели: 37.62432366028436 ; Предсказание модели: 92.59445339109685
Глубина дерева: 10 ; RMSE модели: 37.920256938654454 ; Предсказание модели: 92.61820590011962
Глубина дерева: 11 ; RMSE модели: 38.36551193548183 ; Предсказание м

Результат примерно такой же предыдующего, но с другим методом - LinearRegression

In [258]:
# Сохраним лучшие RMSE всех регионов
best_RMSE_for_all_model_averange_for_all_region.append(best_RMSE_for_all_model_averange[:])
# И предсказания всех регионов
best_predict_for_all_model_averange_for_all_region.append(best_predict_for_all_model_averange[:])

In [259]:
# Очистим, чтобы заполняли новые значения других регионов
best_RMSE_for_all_model_averange.clear()

best_predict_for_all_model_averange.clear()

In [260]:
print(best_predict_for_all_model_averange_for_all_region)
print(best_RMSE_for_all_model_averange_for_all_region)

[[array([ 95.89495185,  77.57258261,  77.89263965, ...,  61.50983303,
       118.18039721, 118.16939229]), array([ 82.66331365,  54.43178616,  29.74875995, ..., 137.87934053,
        83.76196568,  53.95846638]), array([ 93.59963303,  75.10515854,  90.06680936, ...,  99.40728116,
        77.77991248, 129.03241718])], [array([ 97.13034718,  59.67524095,  74.51997296, ...,  64.00436827,
       118.04197451, 118.04197451]), array([ 83.02870355,  53.91376375,  30.11321239, ..., 137.90516594,
        84.01525472,  53.91376375]), array([100.00100405,  58.6402876 ,  99.98247035, ...,  86.77677482,
        66.21706302, 137.28996859])]]
[[37.5794217150813, 0.8930992867756167, 40.02970873393434], [37.52246994257509, 0.7281224686589388, 38.11100425290393]]


In [261]:
%%time

# Примением случайный лес в регрессии

for i in range(0, 3):
    best_model = None
    best_RMSE = 1000
    best_est = 0
    best_depth = 0
    
    print('='*100)
    print('Регион №', i + 1)
    print()

    for est in range(10, 21, 10):
        print('-'*100)
        for depth in range (1, 6):
            # Инициализируем модель RandomForestRegressor с параметрами random_state=12345, n_estimators=est и max_depth=depth
            model = RandomForestRegressor(random_state = 12345, n_estimators = est, max_depth = depth)

            # Обучим модель на тренировочной выборке
            model.fit(features_train[i], target_train[i]) 

            # Получаем предсказания
            predicted_valid = model.predict(features_valid[i])

            RMSE = mean_squared_error(target_valid[i], predicted_valid) ** 0.5
            print('Глубина дерева:', depth, '; RMSE модели:', RMSE, '; Предсказание модели:', predictions_valid.mean())

            if RMSE < best_RMSE:
                best_model = model
                best_RMSE = RMSE
                best_est = est
                best_depth = depth
                best_predictions = predictions_valid
                
    print('-'*100)
    print()
    print("RMSE наилучшей модели на валидационной выборке:", best_RMSE, "Количество деревьев:", best_est, "Глубина дерева:", best_depth)
    print('Лучшие среднее предсказание модели:', best_predictions.mean())

    # Сохраним все значения RMSE
    best_RMSE_for_all_model_averange.append(best_RMSE)
    
    # И предсказания
    best_predict_for_all_model_averange.append(best_predictions)
print('='*100)

Регион № 1

----------------------------------------------------------------------------------------------------
Глубина дерева: 1 ; RMSE модели: 40.33803750452926 ; Предсказание модели: 94.84517637012891
Глубина дерева: 2 ; RMSE модели: 39.0389240716403 ; Предсказание модели: 94.84517637012891
Глубина дерева: 3 ; RMSE модели: 38.194475007108835 ; Предсказание модели: 94.84517637012891
Глубина дерева: 4 ; RMSE модели: 37.73239794392809 ; Предсказание модели: 94.84517637012891
Глубина дерева: 5 ; RMSE модели: 37.460957539612856 ; Предсказание модели: 94.84517637012891
----------------------------------------------------------------------------------------------------
Глубина дерева: 1 ; RMSE модели: 40.34481170358891 ; Предсказание модели: 94.84517637012891
Глубина дерева: 2 ; RMSE модели: 39.039011272860805 ; Предсказание модели: 94.84517637012891
Глубина дерева: 3 ; RMSE модели: 38.1902384961856 ; Предсказание модели: 94.84517637012891
Глубина дерева: 4 ; RMSE модели: 37.7209455896292

Здесь также примерно одианковые результаты, но с небольшими изменениями. Получается, что можно рассмотреть три разные модели по желанию, т.к. они приносят те же примерноые одинаковые результаты 

In [262]:
# Сохраним лучшие RMSE всех регионов
best_RMSE_for_all_model_averange_for_all_region.append(best_RMSE_for_all_model_averange[:])
# И предсказания всех регионов
best_predict_for_all_model_averange_for_all_region.append(best_predict_for_all_model_averange[:])

In [263]:
# Очистим, чтобы заполняли новые значения других регионов
best_RMSE_for_all_model_averange.clear()

best_predict_for_all_model_averange.clear()

In [264]:
print(best_predict_for_all_model_averange_for_all_region)
print(best_RMSE_for_all_model_averange_for_all_region)

[[array([ 95.89495185,  77.57258261,  77.89263965, ...,  61.50983303,
       118.18039721, 118.16939229]), array([ 82.66331365,  54.43178616,  29.74875995, ..., 137.87934053,
        83.76196568,  53.95846638]), array([ 93.59963303,  75.10515854,  90.06680936, ...,  99.40728116,
        77.77991248, 129.03241718])], [array([ 97.13034718,  59.67524095,  74.51997296, ...,  64.00436827,
       118.04197451, 118.04197451]), array([ 83.02870355,  53.91376375,  30.11321239, ..., 137.90516594,
        84.01525472,  53.91376375]), array([100.00100405,  58.6402876 ,  99.98247035, ...,  86.77677482,
        66.21706302, 137.28996859])], [array([112.50001336,  59.13212113,  70.56403956, ...,  86.23879772,
        60.96360518, 147.00586296]), array([112.50001336,  59.13212113,  70.56403956, ...,  86.23879772,
        60.96360518, 147.00586296]), array([112.50001336,  59.13212113,  70.56403956, ...,  86.23879772,
        60.96360518, 147.00586296])]]
[[37.5794217150813, 0.8930992867756167, 40.02970

### 2.4. Напечатаем на экране средний запас предсказанного сырья и RMSE модели

Попробуем составить общую картинку всех регионов и рассмотреть

In [276]:
name_model = ['LinearRegression', 'DecisionTreeRegressor', 'RandomForestRegressor']

for i in range(0, 3):
    print('='*100, end='\n\n')
    print(name_model[i])
    for j in range(0, 3):
        print('Регион №', j + 1)
        print('Лучшие RMSE:', best_RMSE_for_all_model_averange_for_all_region[i][j])
        print('Лучшие средние предсказания:', best_predict_for_all_model_averange_for_all_region[i][j].mean())
    print()

print('='*100)


LinearRegression
Регион № 1
Лучшие RMSE: 37.5794217150813
Лучшие средние предсказания: 92.59256778438035
Регион № 2
Лучшие RMSE: 0.8930992867756167
Лучшие средние предсказания: 68.728546895446
Регион № 3
Лучшие RMSE: 40.02970873393434
Лучшие средние предсказания: 94.96504596800489


DecisionTreeRegressor
Регион № 1
Лучшие RMSE: 37.52246994257509
Лучшие средние предсказания: 92.62631910469075
Регион № 2
Лучшие RMSE: 0.7281224686589388
Лучшие средние предсказания: 68.72973653109962
Регион № 3
Лучшие RMSE: 38.11100425290393
Лучшие средние предсказания: 94.9518475532455


RandomForestRegressor
Регион № 1
Лучшие RMSE: 37.45205379078877
Лучшие средние предсказания: 94.84517637012891
Регион № 2
Лучшие RMSE: 0.7288237012959584
Лучшие средние предсказания: 94.84517637012891
Регион № 3
Лучшие RMSE: 38.41571935965029
Лучшие средние предсказания: 94.84517637012891



### 2.5. Проанализируем результаты и сделаем вывод
Что и требовалось доказать, у второго региона получилось самое маленькое отклонение остальных регионов. А самой оптимальной модели для машинного обучения оказалось LinearRegression, т.к. выигрывает по времени

## 3. Подготовка к расчёту прибыли

### Условия задачи

- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

### 3.1. Все ключевые значения для расчётов сохраним в отдельных переменных

In [266]:
# С помощью машинного обучения выбирали 200 лучшие точки 
best_point_for_product = 200
# Бюджет 10 млрд рублей
budget = 10000000000
# Доход 450 тыс рублей с каждой единицы продукты (объем указывается в тыс баррелей)
income = 450000
# Вероятность убытки 2.5%
probability_of_losses = 0.025

### 3.2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины. Сравним полученный объём сырья со средним запасом в каждом регионе.

In [267]:
# Здесь будут храняться средний запас сырья каждого региона
average_stock_of_raw_products = []

for i in range(0, 3):
    average_stock_of_raw_products.append(full_df[i]['product'].mean())
    print('Ср. запас сырья в ', i + 1, 'регионе: ', round(average_stock_of_raw_products[i], 4), 'млн. баррелей')

Ср. запас сырья в  1 регионе:  92.5 млн. баррелей
Ср. запас сырья в  2 регионе:  68.825 млн. баррелей
Ср. запас сырья в  3 регионе:  95.0 млн. баррелей


In [268]:
# Также выявляем выделенные деньги на постройку одной скважины
budget_one_per_well = budget / best_point_for_product
print('Выделенные деньги на постройку одной скважины:', budget_one_per_well, 'рублей')

Выделенные деньги на постройку одной скважины: 50000000.0 рублей


In [269]:
# Теперь рассчитаем достаточный объем сырья
sufficient_volume_of_raw_products = budget_one_per_well / income
print('Объем каждой скважины (Для безубыточный разработки):', round(sufficient_volume_of_raw_products, 4), 'тыс. баррелей')

Объем каждой скважины (Для безубыточный разработки): 111.1111 тыс. баррелей


In [270]:
# Теперь сравним полученный объем сырья со средним запасом в каждом регионе
# Но в ходе отладки вяснился, что данных очень много и пришлось запихнуть в самом датасете
# Сохраняем полученный объем в каждом регионе в каждом датасете

for i in range(0, 3):
    print('='*100)
    print('Регион №', i + 1)
    full_df[i]['resulting_volume_of_raw_materials'] = full_df[i]['product'] * income
    print(full_df[i]['resulting_volume_of_raw_materials'])
print('='*100)

Регион № 1
0        4.737603e+07
1        3.286699e+07
2        3.836954e+07
3        7.587935e+07
4        6.931649e+07
             ...     
99995    4.983481e+07
99996    5.505608e+07
99997    2.896895e+07
99998    3.331834e+07
99999    6.733496e+07
Name: resulting_volume_of_raw_materials, Length: 100000, dtype: float64
Регион № 2
0        1.430596e+06
1        1.212897e+07
2        6.064484e+07
3        6.207543e+07
4        6.064484e+07
             ...     
99995    2.425793e+07
99996    6.207543e+07
99997    6.207543e+07
99998    1.355956e+07
99999    1.430596e+06
Name: resulting_volume_of_raw_materials, Length: 100000, dtype: float64
Регион № 3
0        1.249140e+07
1        2.523136e+07
2        2.829236e+07
3        5.155778e+07
4        6.732034e+07
             ...     
99995    7.754717e+07
99996    6.243698e+07
99997    7.068604e+07
99998    2.330786e+07
99999    4.624910e+07
Name: resulting_volume_of_raw_materials, Length: 100000, dtype: float64


In [271]:
# Сохранили и теперь находим возврат на инвестиции, т.к. нужно узнать, когда же инвестиционные вложения окупятся
# И эти ROI заполняем в текущий датасет
# Формула ROI = (итоговая прибыль - инвестиции) / инвестици =
# = (resulting_volume_of_raw_materials - budget_one_per_well) / budget_one_per_well

for i in range(0, 3):
    print('='*100)
    print('Регион №', i + 1)
    full_df[i]['return_on_investments'] = (full_df[i]['resulting_volume_of_raw_materials'] - budget_one_per_well) / budget_one_per_well
    # Будет удобнее выразить через процент
    print(round(full_df[i]['return_on_investments'] * 100, 4))
print('='*100)

Регион № 1
0        -5.2479
1       -34.2660
2       -23.2609
3        51.7587
4        38.6330
          ...   
99995    -0.3304
99996    10.1122
99997   -42.0621
99998   -33.3633
99999    34.6699
Name: return_on_investments, Length: 100000, dtype: float64
Регион № 2
0       -97.1388
1       -75.7421
2        21.2897
3        24.1509
4        21.2897
          ...   
99995   -51.4841
99996    24.1509
99997    24.1509
99998   -72.8809
99999   -97.1388
Name: return_on_investments, Length: 100000, dtype: float64
Регион № 3
0       -75.0172
1       -49.5373
2       -43.4153
3         3.1156
4        34.6407
          ...   
99995    55.0943
99996    24.8740
99997    41.3721
99998   -53.3843
99999    -7.5018
Name: return_on_investments, Length: 100000, dtype: float64


In [280]:
# Теперь узнаем, в каком регионе больше всего попадаются отрицательные значения ROI

for i in range(0, 3):
    count_negative_product = 0
    print('='*100)
    print('Регион №', i + 1)

    for ROI in full_df[i]['return_on_investments']:
        # Если в этой точке попадется отрицательные вложения, тогда в копилку попадается
        if(ROI < 0):
            count_negative_product += 1

    # Сохраняем кол-во точек с отрицательными значениями
    print('Кол-во точек с отрицательными значениями:', count_negative_product)
    print('Средние ROI:', round(full_df[i]['return_on_investments'].mean() * 100, 4), end='%\n')

print('='*100)


Регион № 1
Кол-во точек с отрицательными значениями: 63417
Средние ROI: -16.75%
Регион № 2
Кол-во точек с отрицательными значениями: 83463
Средние ROI: -38.0575%
Регион № 3
Кол-во точек с отрицательными значениями: 61822
Средние ROI: -14.5%


### 3.3. Напишем выводы по этапу подготовки расчёта прибыли

Здесь получается, что инвенстировать второй регион не оказалось выгодным условием, т.к. большая часть является убыточной. Лучше всего оказалось третий регион

## 4. Расчёт прибыли и рисков. 

## Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

###  4.1. Выберем скважины с максимальными значениями предсказаний

In [284]:
# Выберем скважину третьего региона, т.к. он лучше остальных по предсказанию
best_well = best_predict_for_all_model_averange_for_all_region[0][2]
print('Лучшие предсказания скважины третьего региона:\n', best_well, '\nИ средние значение предсказания:', round(best_well.mean(), 4))

Лучшие предсказания скважины третьего региона:
 [ 93.59963303  75.10515854  90.06680936 ...  99.40728116  77.77991248
 129.03241718] 
И средние значение предсказания: 94.965


### 4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям

In [292]:
target_value_of_volume_of_raw_materials = best_well.sum()
print('Объем сырья:', round(target_value_of_volume_of_raw_materials, 4), 'тыс. баррелей')

Объем сырья: 2374126.1492 тыс. баррелей


### 4.3. Рассчитаем прибыль для полученного объёма сырья

In [300]:
# Прибыль для полученного объема = (целевое значение объема * доход с одного барреля) - бюджет на разработку скважин в регионе
profit_for_received_volume_of_raw_materials = (target_value_of_volume_of_raw_materials * income) - budget
print('Прибыль:', profit_for_received_volume_of_raw_materials, 'рублей')
print('Или в другом виде:', round(profit_for_received_volume_of_raw_materials / 1000000, 2), 'млн. рублей')

Прибыль: 1058356767140.055 рублей
Или в другом виде: 1058356.77 млн. рублей


In [301]:
# Теперь создадим функцию, которая выполняет аналогично
def profit_for_received(best_predict_well):
    target_value_of_volume_of_raw_materials = best_predict_well.sum()
    print('Объем сырья:', round(target_value_of_volume_of_raw_materials, 4), 'тыс. баррелей')
     
    profit_for_received_volume_of_raw_materials = (target_value_of_volume_of_raw_materials * income) - budget
    print('Прибыль:', profit_for_received_volume_of_raw_materials, 'рублей')
    print('Или в другом виде:', round(profit_for_received_volume_of_raw_materials / 1000000, 2), 'млн. рублей')


In [303]:
for i in range(0, 3):
    print('='*100)
    print('Регион №', i + 1)
    profit_for_received(best_predict_for_all_model_averange_for_all_region[0][i])
print('='*100)

Регион № 1
Объем сырья: 2314814.1946 тыс. баррелей
Прибыль: 1031666387574.2789 рублей
Или в другом виде: 1031666.39 млн. рублей
Регион № 2
Объем сырья: 1718213.6724 тыс. баррелей
Прибыль: 763196152573.7675 рублей
Или в другом виде: 763196.15 млн. рублей
Регион № 3
Объем сырья: 2374126.1492 тыс. баррелей
Прибыль: 1058356767140.055 рублей
Или в другом виде: 1058356.77 млн. рублей


Здесь видно, что третий регион оказался прибыльным по сравнению остальных. Самый низкий показатель по прибыли оказался второй регион

## 5. Посчитайте риски и прибыль для каждого региона

### 5.1. Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли

In [329]:
state = np.random.RandomState(12345)

# сохраните значения 99%-квантилей в переменной values
values = []
for i in range(1000):
    # Чтобы bootstrap работал, нужно работать только с Series, поэтому преобразуем
    best_predict_series = pd.Series(best_predict_for_all_model_averange_for_all_region[0][2], index=target_valid[0].index)
    subsample = best_predict_series.sample(frac=1, replace=True, random_state=state)
    values.append(subsample.quantile(0.99))

values = pd.Series(values)
    
lower = values.quantile(0.05)
upper = values.quantile(0.95)

print(lower)
print(upper)

140.2233236546553
141.55070845082898


<div class="alert alert-block alert-info">

V1: Здесь я тут немного не понял, отсюда можно вычерпнуть с темы 3: "Запуск новой функциональности", урок "Бутстреп для анализа A/B-теста"?
    
</div>

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [ ]  Выполнен шаг 4: посчитаны риски и прибыль
    - [ ]  Проведена процедура *Bootstrap*
    - [ ]  Все параметры бутстрепа соответствуют условию
    - [ ]  Найдены все нужные величины
    - [ ]  Предложен регион для разработки месторождения
    - [ ]  Выбор региона обоснован